In [1]:
import cv2
import numpy as np
from PIL import Image

In [7]:
original_ortho = '/home/caluckal/Desktop/Github/upscaler/validation/original_Ortho.png'
altered_ortho = '/home/caluckal/Desktop/Github/upscaler/validation/downscaled_upscaled_Ortho.png'

In [13]:
# og_img = cv2.imread(original_ortho)
alt_img =cv2.imread(altered_ortho)

In [9]:
#Corner Detection
def corner(image,name):
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)

    #result is dilated for marking the corners, not important
    dst = cv2.dilate(dst,None)

    # Threshold for an optimal value, it may vary depending on the image.
    image[dst>0.01*dst.max()]=[0,0,255]

    cv2.imwrite('harris_{}.png'.format(name),image)

In [10]:
# Edge Detection
def edge(img,name):
    # Convert to graycsale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Blur the image for better edge detection
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0) 
    # Sobel Edge Detection
    sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
    cv2.imwrite('sobelX_{}.png'.format(name),sobelx)
    sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
    cv2.imwrite('sobelY_{}.png'.format(name),sobely)
    sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection
    cv2.imwrite('sobelXY_{}.png'.format(name),sobelxy)
    edges = cv2.Canny(image=img_blur, threshold1=100, threshold2=200) # Canny Edge Detection
    # Display Canny Edge Detection Image
    cv2.imwrite('canny_{}.png'.format(name),edges)


In [11]:
# corner(og_img,'og')

In [14]:
corner(alt_img,'alt')

In [12]:
# edge(og_img,'og')


In [15]:
edge(alt_img,'alt')